<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2 - Singapore Housing Data and Kaggle Challenge

## Contents

### Part 1: 
- [Problem Statement](#Problem-Statement)
- [Data Import & Cleaning](#Data-Import-and-Cleaning)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)

[Part 2:](#part_2.ipynb)
- Data Visualization
- Conclusions and Recommendations

---
## Part 1
---

### Problem Statement

Lorem ipsum

### Aim of Project
Lorem ipsum

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import datasets

df_train = pd.read_csv("../data/train.csv")
df_train.info()
df_train.head()

### Data Cleaning

### Exploratory Data Analysis

### Feature Selection